In [1]:
import pandas as pd
import numpy as np
import cv2
import time
import os

In [2]:
os.path.exists('eu-seg/top_00000.txt')

True

In [3]:
def check_part(loc, no): # loc: eu, ha(jp), th | part: top, bottom, dress | no: file number
    partList = ['top', 'bottom', 'dress']
    partCheck = [0, 0, 0]
    loc = loc + '-seg/'
    for i in range(len(partList)):
        path = loc + '{}_{:05d}.txt'.format(partList[i], no)
        if(os.path.exists(path)):
            partCheck[i] = 1
    return partCheck

In [4]:
def pixel_gathering(loc, part, no):
    list_pos = []
    loc = loc + '-seg/'
    path = loc + '{}_{:05d}.txt'.format(part, no)
    with open(path, 'r') as filestream:
        count = 0
        for line in filestream:
            count = count + 1
            pos_i, pos_j = line.split(', ')
            pos_j = pos_j.replace('\n', '')
            list_pos.append((int(pos_i), int(pos_j)))
            # print('pos_i = {} | pos_j = {}'.format(pos_i, pos_j))
        print('line count = {}'.format(count))
    return list_pos

In [5]:
def count_px(img, posList, color_range):
    
    rgb_count = np.array([[[0 for i in range(0, 256, color_range)] for i in range(0, 256, color_range)] for i in range(0, 256, color_range)])
    
    for [i, j] in posList:
        r_val = img[i, j, 0]
        g_val = img[i, j, 1]
        b_val = img[i, j, 2]

        r_idx = int(r_val/color_range)
        g_idx = int(g_val/color_range)
        b_idx = int(b_val/color_range)

        # print('r: {}, g: {}, b: {}'.format(r_idx, g_idx, b_idx))
        rgb_count[r_idx, g_idx, b_idx] = rgb_count[r_idx, g_idx, b_idx]+1

    count_list = []
    for i in range(color_range):
        for j in range(color_range):
            for k in range(color_range):
                count_list.append(rgb_count[i, j, k])
    
    return count_list

In [6]:
def transform_tab(tab, tab_init, top_n, filename, label):
    top = tab.sort_values('proportion', ascending=False).iloc[0:7].drop('count', axis=1)
    temp = tab_init.copy()
    temp['filename'] = [filename]
    for i in range(1, top_n+1):
        # print(i)
        temp['r_range_{}'.format(i)] = [top.iloc[i-1, 0]]
        temp['g_range_{}'.format(i)] = [top.iloc[i-1, 1]]
        temp['b_range_{}'.format(i)] = [top.iloc[i-1, 2]]
        temp['proportion_{}'.format(i)] = [top.iloc[i-1, 3]]
    temp['label'] = label
    return temp

In [7]:
top7_tab_init = pd.read_csv('Report/top7_count_tap.csv')
top7_tab_top = top7_tab_init.copy()
top7_tab_bottom = top7_tab_init.copy()
top7_tab_dress = top7_tab_init.copy()
count_tab_init = pd.read_csv('Report/count_tab_init.csv')
color_range = 16

In [8]:
st = time.perf_counter()

loc = 'eu'
file_no = 123

part = check_part(loc, file_no)
print(part)

if(part[2]==1):
    pos_dress = pixel_gathering(loc, 'dress', file_no)
    if(part[0]==1):
        pos_top = pixel_gathering(loc, 'top', file_no)
    if(part[1]==1):
        pos_bottom = pixel_gathering(loc, 'bottom', file_no)
else:
    if(part[0]==1):
        pos_top = pixel_gathering(loc, 'top', file_no)
    if(part[1]==1):
        pos_bottom = pixel_gathering(loc, 'bottom', file_no)

    
ed = time.perf_counter()

[1, 1, 1]
line count = 46483
line count = 32339
line count = 32339


In [9]:
a = list(set.union(set(pos_dress), set(pos_top), set(pos_bottom)))
a.sort()

In [10]:
len(a)

46838

In [11]:
# file_loc = location + file_name
st = time.perf_counter()
loc = 'th'
label = 0
for i in range(1000):
    if(loc=='eu'):
        file_loc = 'european-street-fashion-output/test{:05d}.jpg'.format(i)
    elif(loc=='ha'):
        file_loc = 'harajuku-kawaii-street-fashion-output/test{:05d}.jpg'.format(i)
    elif(loc=='th'):
        file_loc = 'thai-street-fashion-output/test{:05d}.jpg'.format(i)
    print(file_loc)
    test = cv2.imread(file_loc)
    count_tab = count_tab_init.copy()
    
    part = check_part(loc, i)
    print(part)

    if(part[2]==1):
        pos_dress = pixel_gathering(loc, 'dress', i)
        pos_top = []
        pos_bottom = []
        if(part[0]==1):
            pos_top = pixel_gathering(loc, 'top', i)
        if(part[1]==1):
            pos_bottom = pixel_gathering(loc, 'bottom', i)
        temp = list(set.union(set(pos_dress), set(pos_top), set(pos_bottom)))
        temp.sort()
        pos_dress = temp.copy()
        
        count_list = count_px(test, pos_dress, color_range)
        count_tab['count'] = count_list
        count_tab['proportion'] = count_tab['count']/sum(count_tab['count'])
        temp_top7_tab = transform_tab(count_tab, top7_tab_init, 7, file_loc, label)
        top7_tab_dress = pd.concat([top7_tab_dress, temp_top7_tab], axis=0)
        
    else:
        if(part[0]==1):
            pos_top = pixel_gathering(loc, 'top', i)
            count_list = count_px(test, pos_top, color_range)
            count_tab['count'] = count_list
            count_tab['proportion'] = count_tab['count']/sum(count_tab['count'])
            temp_top7_tab = transform_tab(count_tab, top7_tab_init, 7, file_loc, label)
            top7_tab_top = pd.concat([top7_tab_top, temp_top7_tab], axis=0)

        if(part[1]==1):
            pos_bottom = pixel_gathering(loc, 'bottom', i)

            count_list = count_px(test, pos_bottom, color_range)
            count_tab['count'] = count_list
            count_tab['proportion'] = count_tab['count']/sum(count_tab['count'])
            temp_top7_tab = transform_tab(count_tab, top7_tab_init, 7, file_loc, label)
            top7_tab_bottom = pd.concat([top7_tab_bottom, temp_top7_tab], axis=0)
        
    
ed = time.perf_counter() 

thai-street-fashion-output/test00000.jpg
[0, 1, 0]
line count = 35099
thai-street-fashion-output/test00001.jpg
[0, 0, 0]
thai-street-fashion-output/test00002.jpg
[0, 0, 0]
thai-street-fashion-output/test00003.jpg
[1, 1, 0]
line count = 30771
line count = 24360
thai-street-fashion-output/test00004.jpg
[1, 1, 0]
line count = 23040
line count = 27972
thai-street-fashion-output/test00005.jpg
[1, 0, 0]
line count = 14687
thai-street-fashion-output/test00006.jpg
[1, 1, 0]
line count = 30557
line count = 64664
thai-street-fashion-output/test00007.jpg
[1, 0, 0]
line count = 63952
thai-street-fashion-output/test00008.jpg
[0, 0, 0]
thai-street-fashion-output/test00009.jpg
[1, 1, 0]
line count = 5035
line count = 1639
thai-street-fashion-output/test00010.jpg
[0, 1, 0]
line count = 1478
thai-street-fashion-output/test00011.jpg
[0, 0, 0]
thai-street-fashion-output/test00012.jpg
[1, 1, 0]
line count = 22897
line count = 9944
thai-street-fashion-output/test00013.jpg
[0, 0, 1]
line count = 32525
thai-

thai-street-fashion-output/test00122.jpg
[0, 1, 0]
line count = 19463
thai-street-fashion-output/test00123.jpg
[0, 0, 0]
thai-street-fashion-output/test00124.jpg
[0, 0, 0]
thai-street-fashion-output/test00125.jpg
[1, 1, 0]
line count = 146611
line count = 86114
thai-street-fashion-output/test00126.jpg
[0, 0, 0]
thai-street-fashion-output/test00127.jpg
[0, 0, 0]
thai-street-fashion-output/test00128.jpg
[0, 0, 0]
thai-street-fashion-output/test00129.jpg
[0, 0, 0]
thai-street-fashion-output/test00130.jpg
[0, 1, 0]
line count = 829478
thai-street-fashion-output/test00131.jpg
[0, 0, 0]
thai-street-fashion-output/test00132.jpg
[0, 0, 0]
thai-street-fashion-output/test00133.jpg
[1, 0, 0]
line count = 68763
thai-street-fashion-output/test00134.jpg
[1, 1, 0]
line count = 62100
line count = 44885
thai-street-fashion-output/test00135.jpg
[1, 1, 0]
line count = 36440
line count = 30377
thai-street-fashion-output/test00136.jpg
[0, 0, 0]
thai-street-fashion-output/test00137.jpg
[0, 0, 0]
thai-street

line count = 65218
thai-street-fashion-output/test00242.jpg
[0, 1, 0]
line count = 44091
thai-street-fashion-output/test00243.jpg
[1, 0, 1]
line count = 78881
line count = 34777
thai-street-fashion-output/test00244.jpg
[1, 0, 0]
line count = 55603
thai-street-fashion-output/test00245.jpg
[1, 0, 0]
line count = 62395
thai-street-fashion-output/test00246.jpg
[0, 0, 0]
thai-street-fashion-output/test00247.jpg
[0, 0, 0]
thai-street-fashion-output/test00248.jpg
[0, 0, 0]
thai-street-fashion-output/test00249.jpg
[0, 0, 0]
thai-street-fashion-output/test00250.jpg
[0, 0, 0]
thai-street-fashion-output/test00251.jpg
[1, 0, 1]
line count = 54871
line count = 23421
thai-street-fashion-output/test00252.jpg
[1, 1, 0]
line count = 27549
line count = 40695
thai-street-fashion-output/test00253.jpg
[1, 1, 0]
line count = 60273
line count = 27308
thai-street-fashion-output/test00254.jpg
[1, 1, 0]
line count = 3056
line count = 2782
thai-street-fashion-output/test00255.jpg
[1, 1, 0]
line count = 162528
li

line count = 33431
thai-street-fashion-output/test00363.jpg
[1, 1, 0]
line count = 22982
line count = 37953
thai-street-fashion-output/test00364.jpg
[0, 0, 0]
thai-street-fashion-output/test00365.jpg
[0, 0, 0]
thai-street-fashion-output/test00366.jpg
[0, 0, 0]
thai-street-fashion-output/test00367.jpg
[0, 0, 0]
thai-street-fashion-output/test00368.jpg
[0, 0, 0]
thai-street-fashion-output/test00369.jpg
[0, 0, 0]
thai-street-fashion-output/test00370.jpg
[0, 0, 0]
thai-street-fashion-output/test00371.jpg
[0, 0, 0]
thai-street-fashion-output/test00372.jpg
[0, 1, 0]
line count = 8689
thai-street-fashion-output/test00373.jpg
[0, 1, 0]
line count = 2264
thai-street-fashion-output/test00374.jpg
[1, 1, 0]
line count = 7304
line count = 13758
thai-street-fashion-output/test00375.jpg
[0, 0, 0]
thai-street-fashion-output/test00376.jpg
[0, 0, 0]
thai-street-fashion-output/test00377.jpg
[1, 1, 0]
line count = 51390
line count = 208985
thai-street-fashion-output/test00378.jpg
[1, 1, 0]
line count = 27

line count = 63846
thai-street-fashion-output/test00489.jpg
[0, 0, 0]
thai-street-fashion-output/test00490.jpg
[0, 0, 0]
thai-street-fashion-output/test00491.jpg
[1, 0, 0]
line count = 39751
thai-street-fashion-output/test00492.jpg
[0, 0, 1]
line count = 20223
thai-street-fashion-output/test00493.jpg
[1, 1, 0]
line count = 53248
line count = 19454
thai-street-fashion-output/test00494.jpg
[1, 0, 0]
line count = 142487
thai-street-fashion-output/test00495.jpg
[1, 1, 0]
line count = 164447
line count = 70563
thai-street-fashion-output/test00496.jpg
[1, 1, 0]
line count = 157259
line count = 57435
thai-street-fashion-output/test00497.jpg
[0, 1, 0]
line count = 18805
thai-street-fashion-output/test00498.jpg
[1, 0, 0]
line count = 16650
thai-street-fashion-output/test00499.jpg
[1, 0, 0]
line count = 8160
thai-street-fashion-output/test00500.jpg
[0, 0, 0]
thai-street-fashion-output/test00501.jpg
[0, 1, 0]
line count = 89002
thai-street-fashion-output/test00502.jpg
[0, 1, 0]
line count = 32144

[0, 0, 0]
thai-street-fashion-output/test00925.jpg
[0, 0, 0]
thai-street-fashion-output/test00926.jpg
[0, 0, 0]
thai-street-fashion-output/test00927.jpg
[0, 0, 0]
thai-street-fashion-output/test00928.jpg
[0, 0, 0]
thai-street-fashion-output/test00929.jpg
[0, 0, 0]
thai-street-fashion-output/test00930.jpg
[0, 0, 0]
thai-street-fashion-output/test00931.jpg
[0, 0, 0]
thai-street-fashion-output/test00932.jpg
[0, 0, 0]
thai-street-fashion-output/test00933.jpg
[0, 0, 0]
thai-street-fashion-output/test00934.jpg
[0, 0, 0]
thai-street-fashion-output/test00935.jpg
[0, 0, 0]
thai-street-fashion-output/test00936.jpg
[0, 0, 0]
thai-street-fashion-output/test00937.jpg
[0, 0, 0]
thai-street-fashion-output/test00938.jpg
[0, 0, 0]
thai-street-fashion-output/test00939.jpg
[0, 0, 0]
thai-street-fashion-output/test00940.jpg
[0, 0, 0]
thai-street-fashion-output/test00941.jpg
[0, 0, 0]
thai-street-fashion-output/test00942.jpg
[0, 0, 0]
thai-street-fashion-output/test00943.jpg
[0, 0, 0]
thai-street-fashion-o

In [12]:
top7_tab_top

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,...,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,thai-street-fashion-output/test00003.jpg,192-207,192-207,192-207,0.109649,192-207,160-175,112-127,0.083260,176-191,...,0.058269,208-223,176-191,128-143,0.044912,208-223,192-207,176-191,0.030093,0
0,thai-street-fashion-output/test00004.jpg,48-63,48-63,48-63,0.163542,64-79,64-79,64-79,0.090538,64-79,...,0.056163,48-63,48-63,64-79,0.052170,80-95,64-79,80-95,0.043490,0
0,thai-street-fashion-output/test00005.jpg,192-207,176-191,160-175,0.266698,208-223,192-207,176-191,0.167291,208-223,...,0.033295,208-223,176-191,128-143,0.016545,128-143,128-143,128-143,0.014094,0
0,thai-street-fashion-output/test00006.jpg,192-207,208-223,208-223,0.219459,192-207,208-223,224-239,0.084432,208-223,...,0.043362,192-207,192-207,208-223,0.038682,208-223,208-223,208-223,0.037635,0
0,thai-street-fashion-output/test00007.jpg,16-31,16-31,16-31,0.436108,32-47,32-47,32-47,0.149487,48-63,...,0.025410,0-15,0-15,0-15,0.023033,64-79,64-79,64-79,0.020187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,thai-street-fashion-output/test00494.jpg,16-31,16-31,16-31,0.455957,32-47,16-31,16-31,0.168535,16-31,...,0.055219,32-47,32-47,32-47,0.039260,16-31,0-15,0-15,0.038972,0
0,thai-street-fashion-output/test00495.jpg,112-127,64-79,240-255,0.092674,96-111,48-63,240-255,0.090637,112-127,...,0.054303,96-111,64-79,240-255,0.041922,80-95,32-47,240-255,0.022737,0
0,thai-street-fashion-output/test00496.jpg,144-159,112-127,48-63,0.030415,128-143,112-127,64-79,0.028189,128-143,...,0.024711,128-143,96-111,32-47,0.024495,144-159,112-127,64-79,0.024094,0
0,thai-street-fashion-output/test00498.jpg,32-47,32-47,32-47,0.416637,48-63,32-47,32-47,0.221922,48-63,...,0.045165,64-79,64-79,64-79,0.015676,16-31,16-31,16-31,0.014414,0


In [13]:
top7_tab_bottom

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,...,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,thai-street-fashion-output/test00000.jpg,32-47,48-63,176-191,0.308014,32-47,48-63,192-207,0.098949,32-47,...,0.054845,32-47,48-63,160-175,0.041084,16-31,32-47,144-159,0.036611,0
0,thai-street-fashion-output/test00003.jpg,48-63,64-79,96-111,0.244212,32-47,64-79,96-111,0.084360,32-47,...,0.062438,48-63,80-95,112-127,0.061823,48-63,64-79,112-127,0.043432,0
0,thai-street-fashion-output/test00004.jpg,48-63,48-63,48-63,0.531496,48-63,48-63,32-47,0.106571,64-79,...,0.028493,32-47,32-47,32-47,0.015659,96-111,96-111,80-95,0.014908,0
0,thai-street-fashion-output/test00006.jpg,48-63,48-63,48-63,0.448580,32-47,32-47,32-47,0.340282,16-31,...,0.026181,48-63,64-79,48-63,0.011274,64-79,64-79,64-79,0.010423,0
0,thai-street-fashion-output/test00009.jpg,112-127,96-111,112-127,0.066504,80-95,80-95,96-111,0.057352,176-191,...,0.048200,192-207,192-207,192-207,0.040268,128-143,112-127,128-143,0.037218,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,thai-street-fashion-output/test00495.jpg,144-159,96-111,64-79,0.131259,160-175,112-127,80-95,0.111276,144-159,...,0.043720,128-143,96-111,64-79,0.035585,160-175,112-127,96-111,0.031532,0
0,thai-street-fashion-output/test00496.jpg,128-143,112-127,64-79,0.030800,144-159,128-143,80-95,0.027649,112-127,...,0.022060,96-111,80-95,48-63,0.018856,96-111,80-95,32-47,0.017307,0
0,thai-street-fashion-output/test00497.jpg,0-15,0-15,0-15,0.288328,16-31,16-31,16-31,0.138687,16-31,...,0.023504,0-15,0-15,16-31,0.021218,16-31,16-31,0-15,0.019250,0
0,thai-street-fashion-output/test00501.jpg,16-31,0-15,0-15,0.407687,0-15,0-15,0-15,0.170288,32-47,...,0.033561,144-159,176-191,208-223,0.030718,48-63,16-31,16-31,0.023977,0


In [14]:
top7_tab_dress

,filename,r_range_1,g_range_1,b_range_1,proportion_1,r_range_2,g_range_2,b_range_2,proportion_2,r_range_3,...,proportion_5,r_range_6,g_range_6,b_range_6,proportion_6,r_range_7,g_range_7,b_range_7,proportion_7,label
0,thai-street-fashion-output/test00013.jpg,240-255,192-207,144-159,0.433605,224-239,176-191,128-143,0.139247,240-255,...,0.039508,224-239,176-191,112-127,0.033912,240-255,176-191,128-143,0.029731,0
0,thai-street-fashion-output/test00019.jpg,16-31,16-31,16-31,0.066526,224-239,208-223,208-223,0.045349,208-223,...,0.030237,240-255,240-255,240-255,0.026123,224-239,224-239,224-239,0.020174,0
0,thai-street-fashion-output/test00054.jpg,48-63,64-79,80-95,0.051433,48-63,48-63,80-95,0.047170,16-31,...,0.037453,64-79,80-95,112-127,0.032422,64-79,48-63,48-63,0.031834,0
0,thai-street-fashion-output/test00063.jpg,176-191,128-143,144-159,0.108213,176-191,112-127,144-159,0.068735,96-111,...,0.041593,160-175,96-111,128-143,0.040536,144-159,80-95,112-127,0.039831,0
0,thai-street-fashion-output/test00072.jpg,32-47,16-31,16-31,0.124104,16-31,0-15,0-15,0.102323,48-63,...,0.043322,48-63,32-47,16-31,0.043136,64-79,48-63,32-47,0.036275,0
0,thai-street-fashion-output/test00107.jpg,16-31,32-47,48-63,0.169140,32-47,48-63,64-79,0.162228,16-31,...,0.051261,16-31,16-31,16-31,0.045831,32-47,64-79,80-95,0.043384,0
0,thai-street-fashion-output/test00147.jpg,192-207,160-175,240-255,0.108925,176-191,144-159,240-255,0.085058,208-223,...,0.022764,144-159,112-127,240-255,0.022456,192-207,144-159,240-255,0.020470,0
0,thai-street-fashion-output/test00156.jpg,16-31,16-31,16-31,0.179952,32-47,16-31,32-47,0.116010,32-47,...,0.060221,48-63,32-47,16-31,0.039064,48-63,32-47,48-63,0.030372,0
0,thai-street-fashion-output/test00164.jpg,64-79,48-63,32-47,0.242806,48-63,32-47,16-31,0.137913,48-63,...,0.042969,64-79,48-63,48-63,0.041431,80-95,64-79,48-63,0.028275,0
0,thai-street-fashion-output/test00191.jpg,32-47,16-31,16-31,0.480785,240-255,240-255,240-255,0.068684,240-255,...,0.038430,16-31,16-31,16-31,0.032161,224-239,224-239,224-239,0.022895,0


In [15]:
time_in_sec = ed-st
time_h = int(time_in_sec/3600)
time_m = int((time_in_sec-60*time_h)/60)
time_s = time_in_sec-3600*time_h-60*time_m

In [16]:
print('Process Time: {} hr(s). {} min(s) {:.6f} s.'.format(time_h, time_m, time_s))

Process Time: 0 hr(s). 4 min(s) 50.898078 s.


In [17]:
print('Process Time: {:.6f} s.'.format(ed-st))

Process Time: 290.898078 s.


In [18]:
st

2.6504175

In [19]:
ed

293.5484958